# REPORTE DE ANÁLISIS DE COSTOS
### Empresa Biotech Explorer Corp.


El objetivo de este reporte es ofrecer un análisis sobre el costo de disponibilizar una solución para el análisis de imágenes de secuecniación genética en la nuble de Amazon (AWS). Este análisis se realiza de manera específica para cada componente de la arquietctura, y al final de manera general para entender costos totales. Adicionalemente se podrán modificar los requerimientos técnicos para realizar una proyección de los costos según modificaciones de parámetros de números de instancias y capacidad de almacenamiento.


- Los componentes de la arquietctura son los siguientes:
    - Servidor EC2 de modelos (engine) para el procesamiento de secuencias de ADN
    - Servidor de bases de datos Postgresql RDS 
    - Repositorio S3 de imágenes de secuenciación genética

### Contexto
Biotech Explorer Corp. es una pequeña empresa (StartUp) del sector farmacuéutico que realiza análsis de imágenes de secuenciación genética en búsqueda de patrones genéticos que permitan construir vacunas contra enfermedades. Para esto desarrolla modelos de Machine Learning (computer vision) que se encargan de procesar un gran banco de imágenes de secuenciación genética para extraer datos e información que almacena en un servidor para posteriormente realizar análsisi y otros procesos para extraer patrones genpeticos valiosos para la construcción de vacunas. Estos procesos se realizan sistemáticamente y con cargas constantes durante todo el día, dado que la empresa dispone de una infraestructura dedicada solo a estos procesos y que no intervienen ni afectan los procesos de ventas, usuarios y otros procesos sustantivos como nómina, contabilidad, entre otros. La empresa tiene **ingresos mensuales** operacionales. por US$ 5MM, con un margen operacional del 10%, o sea US$ 500K mensuales, los que se utilizan para el pago de nóminas para los 100 empleados , gastos administrativos y operacionales, disponiendo aproximadamente el 1% de este presupuesto mensual para el pago de infraestructura tecnológica destinada a la investigación de nuevos patrones genéticos para el desarrollo de vacunas; es decir el presupuesto mensual para costos de operación tecnológica es de máximo US$ 5K.

In [19]:
import pandas as pd

### 1. Análisis de costos servidor EC2
Teniendo en cuenta que se realizarán procesos sistemáticos, con cargas de trabajo constantes, y que los procesos de análisis de las imágenes de secuenciación de ADN se realizará en memoria, por la naturaleza de este proceso se disponen instancias **dedicadas** para estos modelos, con **16vCPUs** para potenciar el uso de paralelismo, y con una memoria RAM de **32 GB**. A pesar que no se contemplan picos programados, sino que la carga esperada (en un percentil 95) será constante. Adicionalmente esta instancia **no requerirá** de almacenamiento local ya que las imágenes y la información extraída se persisten en servidores (storages) dedicados para esto.

Teniendo en cuentas las características definidas y los precios estimados según [la caulculadora de precios de AWS](https://calculator.aws/), la tabla de precios **de la propuesta** es la siguiente:

In [20]:
ec2_princing= 0.408
ec2_instance = 1
ec2 = {'Recurso IT': 'Servidor de modelos Python',
                'Servicio Cloud': 'EC2',
                'Region': 'Norte de Virginia',
                'Especificación' : 'a1.4xlarge\n(32 GiB memory\t16 vCPUs)',
                'Instancias dedicadas': 'Sí',
                'Instancias' : ec2_instance,
                'Precio por hora on-demand': ec2_princing,
                'Cant. horas al mes': 730,
                'Fee mensual (USD)': 1460,
                'Costo mes (USD)':1757.84}
df_ec2 = pd.DataFrame([ec2])
df_ec2

,Recurso IT,Servicio Cloud,Region,Especificación,Instancias dedicadas,Instancias,Precio por hora on-demand,Cant. horas al mes,Fee mensual (USD),Costo mes (USD)
0,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,1,0.408,730,1460,1757.84


#### 1.1 Proyección de aumento de costos dado aumento de instancias
Proyectando un aumento en número de instancias hasta 10, el costo mensual estimado (proyectado) sería el siguiente.
> Nota: si se desea se puede modificar este número

In [21]:
ec2_instance_forecasting = 10

def ec2_fee_mensual(df):
        horas_mes = 730
        return horas_mes*2
    
def ec2_month_pricing(df):
    return (df['Instancias'] * ec2_princing * 730) + (df['Fee mensual (USD)'])


def row_repeat(df, repeat_num, index_column_increment_1, index_column_increment_2 = None, increment_1 = 1, increment_2 = 1):
    fila_lista = df.iloc[0].values.tolist()
    for i in range(1, repeat_num+1):
        df.loc[len(df)] = fila_lista
        fila_lista[index_column_increment_1] = fila_lista[index_column_increment_1]  + increment_1
        if index_column_increment_2 is not None:
            fila_lista[index_column_increment_2] = fila_lista[index_column_increment_2]  + increment_2
    df = df.drop(index=0).reset_index(drop=True)
    return df

In [22]:
df_ec2_forecasting = df_ec2.copy()
df_ec2_forecasting = row_repeat(df_ec2_forecasting, ec2_instance_forecasting, 5)
df_ec2_forecasting['Fee mensual (USD)'] = df_ec2_forecasting.apply(ec2_fee_mensual, axis=1)
df_ec2_forecasting['Costo mes (USD)'] = df_ec2_forecasting.apply(ec2_month_pricing, axis=1)

df_ec2_forecasting

,Recurso IT,Servicio Cloud,Region,Especificación,Instancias dedicadas,Instancias,Precio por hora on-demand,Cant. horas al mes,Fee mensual (USD),Costo mes (USD)
0,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,1,0.408,730,1460,1757.84
1,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,2,0.408,730,1460,2055.68
2,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,3,0.408,730,1460,2353.52
3,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,4,0.408,730,1460,2651.36
4,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,5,0.408,730,1460,2949.20
5,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,6,0.408,730,1460,3247.04
6,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,7,0.408,730,1460,3544.88
7,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,8,0.408,730,1460,3842.72
8,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,9,0.408,730,1460,4140.56
9,Servidor de modelos Python,EC2,Norte de Virginia,a1.4xlarge\n(32 GiB memory\t16 vCPUs),Sí,10,0.408,730,1460,4438.40


Desde la óptica de costos mensuales, los modelos ocuparían el 35% del presupuesto mensual, con **US$ 1757.84**. El análisis de la proyección indica un aumento por encima del presupuesto mensual, lo cual sería en principis inviable y habría que analizar con el departamento financiero si son costos que se pudisen asumir. De todas maneras escalar la solución a 2 instancias llevaría los costos de EC2 a **US$ 2055.68** lo cual estaría dentro de lo presupuestado, en caso de una contigencia que implicara escalar la solución a dos instancias.

### 2. Análisis de costos storage S3 de impágenes de secuenciación de ADN

> Assumptions: a) se asumirán imágenes fotográficas de alta resolución de secuenciación de ADN con un peso de hasta **15 MB**, las que se producidas por equipos tecnológicos especializados capaz de procesar muestras de ADN animal y vegetal; b) los estudios genéticos se realizan sobre poblaciones de cientos de miles de individuos, nunca mayor a 5 millones; c) una vez que se ha terminado con el estudio de una población, la imágenes de secuenciación de ADN dejan de ser relevantes para la empresa y pasan a almacenarse en un storage "en frío" (backup) que está a cargo de otra área tercerizada (especializada en custodiar material e información genética); y d) las imágenes ya están almacenadas en instancias S3 y son proveedidas por una empresa aliada (esto reduce costos de carga hacia S3)

Teniendo en cuenta el peso de las imágenes de secuenciación genética, y estimando para poblaciones muy grande (en el orden de los 12 millones de individuos a estudiar), se estima una capacidad de almacenamiento máxima de aproximadamente **72TB**

Segpun el esquema de precios para una instancia **S3 estándar** se distribuyen los precios de la siguiente manera:
- los primero 50TB a US$ 0.023 x TB
- los próximos 450 TB a US$ 0.022 x TB
- los siguientes TB a US$ 0.021 x TB 
Y así va reduciéndose el precio

Para este caso en particular, solo interesan los dos primeros niveles de pricing:
- los primero 50TB a US$ 0.023 x TB
- los próximos 22 TB a US$ 0.022 x TB

El análisis de costos para este componente de infraestructura, que es considerado muy crítico ya que es el que almacena las imágenes a analizar, quedaría de la siguiente manera.

In [27]:
s3_l1_princing= 0.023
s3_l2_princing= 0.022
storage_tb = 72
storage_gb = storage_tb  * 1024

s3 = {'Recurso IT': 'Repositorio de imágenes',
                'Servicio Cloud': 'S3',
                'Region': 'Norte de Virginia',
                'Especificación' : 'S3 Standard',
                'Instancias dedicadas': 'Sí',
                'Almacenamiento (TB)' : storage_tb,
                'Almacenamiento (GB)' : storage_gb,
                'Precio primeros 50TB': s3_l1_princing * (50 * 1024),
                'Precio resto TB': s3_l2_princing * (storage_tb-50) * 1024,
                'Costo mes (USD)': 1673.22}
df_s3 = pd.DataFrame([s3])
df_s3

,Recurso IT,Servicio Cloud,Region,Especificación,Instancias dedicadas,Almacenamiento (TB),Almacenamiento (GB),Precio primeros 50TB,Precio resto TB,Costo mes (USD)
0,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,72,73728,1177.6,495.616,1673.22


Para una capacidad de almacenamiento de 72TB, lo que supone una capacidad para almacenar unos 5 millones de impagenes de secuecniación de cadenas de ADN, el costo mensual es de aproximadamente **US$ 1673**, lo que sumado al costo de EC2 (US$ 1757) representa casi el 69% del presupuesto mensual destinado.

Realizando una proyección de costos aumentando la capacidad de almacenamiento a casi el doble de lo previsto, quedaría de la siguiente manera.

In [32]:
def s3_diff_tb_pricing(df):
    diff = df['Almacenamiento (TB)'] - 50
    if diff > 0 and diff < 460:
        return diff * 1024 * s3_l2_princing
    else:
        0



def s3_month_pricing(df):
    return df['Precio primeros 50TB'] + df['Precio resto TB']


s3_tb_forecasting = 10
df_s3_forecasting = df_s3.copy()

df_s3_forecasting = row_repeat(df_s3_forecasting, s3_tb_forecasting, 10,5)
df_s3_forecasting['Precio resto TB'] = df_s3_forecasting.apply(s3_diff_tb_pricing, axis=1)
df_s3_forecasting['Costo mes (USD)'] = df_s3_forecasting.apply(s3_month_pricing, axis=1)


df_s3_forecasting
    

,Recurso IT,Servicio Cloud,Region,Especificación,Instancias dedicadas,Almacenamiento (TB),Almacenamiento (GB),Precio primeros 50TB,Precio resto TB,Costo mes (USD)
0,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,72,73728,1177.6,495.616,1673.216
1,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,74,73728,1177.6,540.672,1718.272
2,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,76,73728,1177.6,585.728,1763.328
3,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,78,73728,1177.6,630.784,1808.384
4,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,80,73728,1177.6,675.840,1853.440
5,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,82,73728,1177.6,720.896,1898.496
6,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,84,73728,1177.6,765.952,1943.552
7,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,86,73728,1177.6,811.008,1988.608
8,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,88,73728,1177.6,856.064,2033.664
9,Repositorio de imágenes,S3,Norte de Virginia,S3 Standard,Sí,90,73728,1177.6,901.120,2078.720
